In [40]:
import pandas as pd
from datetime import datetime


# Crawl data from yahoo finance

In [41]:

start_date = datetime(2012,4,1).strftime('%s')
end_date = datetime(2016,4,1).strftime('%s')
stock = "D05.SI"

def getCrumb(ticker):
    URL  = "https://finance.yahoo.com/quote/%s/history" % ticker
    import requests 
    import re
    r = requests.get(url = URL) 
    print(r)
    content = str(r.content)
    r1 = re.findall(r"CrumbStore\":\{\"crumb\":\"[A-Za-z0-9.\\\-]+\"",content)[0]
    crumb = r1.replace("CrumbStore\":{\"crumb\":\"","").replace('"',"")
    return crumb,r.cookies

def get_data(data_type, symbol):
    import requests 
    import re
    from io import StringIO
    api_url = "https://query1.finance.yahoo.com/v7/finance/download/{0}?period1={1}&period2={2}&interval={3}&events={4}&crumb={5}"
    (crumb, cookies)=getCrumb(symbol)
    api_url = api_url.format(symbol,start_date,end_date,"1d", data_type, crumb)
    r = requests.get(url = api_url, cookies = cookies.get_dict(".yahoo.com"))  
    print(r)
    content = str(r.content).replace(",",'","').replace("\\n",'"\r\n"').replace("b\'","").replace("'","")
    res = '"'+content+'"'
    df = pd.read_csv(StringIO(res), sep=",",engine='python')
    return df

In [42]:
price = get_data("history", stock)
price.rename(columns={'Date':'date','Open':'open','High':'high',
                      'Low':'low','Close':'close','Adj Close':'adjClose','Volume':'vol'}, inplace=True)
price["date"]=pd.to_datetime(price['date'])
price = price.set_index(["date"])
price

<Response [200]>
<Response [200]>


,open,high,low,close,adjClose,vol
date,,,,,,
2012-04-02,14.18,14.18,14.18,14.18,10.491443,0
2012-04-03,13.70,13.85,13.58,13.79,10.202891,25932000
2012-04-04,13.69,13.73,13.48,13.48,9.973529,16907000
2012-04-05,13.48,13.48,13.26,13.34,9.869947,12255000
2012-04-09,13.24,13.32,13.23,13.27,9.818156,3702000
...,...,...,...,...,...,...
2016-03-24,15.49,15.49,15.22,15.34,12.966033,6561500
2016-03-28,15.32,15.32,15.12,15.17,12.822343,4226700
2016-03-29,15.30,15.30,15.13,15.18,12.830796,4000200


In [43]:
dividend = get_data("div", stock)
dividend.rename(columns={'Date':'date','Dividends':'dividend'}, inplace=True)
dividend["date"]=pd.to_datetime(dividend['date'])
dividend = dividend.set_index(["date"])
dividend

<Response [200]>
<Response [200]>


,dividend
date,
2013-05-13,0.28
2014-05-12,0.30
2012-05-09,0.28
2012-08-15,0.28
2014-08-13,0.28
2015-07-31,0.30
2013-08-15,0.28
2015-04-27,0.30


# Get data from postgres Database [SKIP THIS!]

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.dates as mdates
from scipy import stats
import datetime as dt
import psycopg2

connection = psycopg2.connect(user = "postgres", password = "admin", host = "127.0.0.1",port = "5432", database = "postgres")

def close_conn():
    if(connection):
        cursor.close()
        connection.close()

def println(str):
    show_each_tries= True
    if(show_each_tries):
        print(str)

In [ ]:
stock = "D05.SI"
sdate='2017-04-01'
edate='2020-03-27'
price = pd.read_sql_query("SELECT date as Date, open as Open, close as Close  FROM finance.stock_price where stock = '"+stock+"' and date between '"+sdate+"' and '"+edate+"' order by date asc",con=connection)
price["date"]=pd.to_datetime(price['date'])
price = price.set_index(["date"])
price

In [ ]:
dividend = pd.read_sql_query("SELECT * FROM finance.stock_dividend where stock = '"+stock+"' and date between '"+sdate+"' and '"+edate+"' order by date asc",con=connection)
dividend["date"]=pd.to_datetime(dividend['date'])
dividend = dividend.set_index(["date"])
dividend

# Calculate gross return

In [45]:
from datetime import timedelta, date

df = pd.DataFrame([], columns = ['Date' , 'Gross', 'GrossPct', 'Close', 'ClosePct']) 
init_close = -1
init_amt = 10000
amount = init_amt
shares = 0
fee = 0.001
curDate = price.index.min()
last_close = 0
last_open = 0
while True:
    if(curDate in price.index):
        curRec = price.loc[[curDate]]
        curOpen = float(curRec["open"].values[0])
        curClose = float(curRec["close"].values[0])
        last_open = curOpen
        last_close = curClose
        if(init_close==-1):
            init_close = last_close
        if(amount>0):
            #buy shares
            share_can_buy = (int)(amount/(curOpen*(1+fee)))
            amount = amount - share_can_buy*(curOpen*(1+fee))
            shares = shares + share_can_buy
    if(curDate in dividend.index):
        curDiv = dividend.loc[[curDate]]["dividend"].values[0]
        amount = amount + (shares*float(curDiv))
    if(curDate == price.index.max()):
        break
    gross = shares*curClose + amount
    df = df.append({'Date' : curDate , 'Gross' : round(gross,2), 'GrossPct' :  round(((gross-init_amt)/init_amt)*100,2), 'Close': curClose, 'ClosePct': round(((curClose-init_close)/init_close)*100,2)} , ignore_index=True)
    curDate = curDate + timedelta(days=1)

# Plot line chart

In [46]:
def plot_chart(data, height, width, cols = []):
    import plotly.graph_objs as go 
    from datetime import datetime
    from ipywidgets import interact, interactive, fixed, interact_manual
    import ipywidgets as widgets
    from plotly.subplots import make_subplots
    
    df = data.set_index('Date')
    fig = go.FigureWidget(make_subplots(rows=1))
    
    
    for col in cols:
        fig.add_trace(
            go.Scattergl(x=list(df.index), y=list(df[col]),
            name = col),
            row=1, col=1
        )
    
    df["const"] = 0
    
    fig.add_trace(
            go.Scattergl(x=list(df.index), y=list(df["const"]), line = dict(color='grey', width=1, dash='dash'),
            name = col),
            row=1, col=1
    )
    fig.layout.xaxis=dict(
            anchor='x',
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label='1m',
                         step='month',
                         stepmode='backward'),
                    dict(count=6,
                         label='6m',
                         step='month',
                         stepmode='backward'),
                    dict(count=1,
                        label='YTD',
                        step='year',
                        stepmode='todate'),
                    dict(count=1,
                        label='1y',
                        step='year',
                        stepmode='backward'),
                    dict(step='all')
                ])
            ),
            type='date'
        )
    fig.layout.yaxis=dict(
            domain=[0, 1]
        )
    fig['layout'].update(height=height, width=width) 
    return fig

In [47]:
#plot_chart(df, 500, 900, ['Gross', 'GrossPct', 'Close', 'ClosePct'])
plot_chart(df, 500, 900, ['GrossPct', 'ClosePct'])

FigureWidget({
    'data': [{'name': 'GrossPct',
              'type': 'scattergl',
              'uid': '1879…